In [7]:
import openmatrix as omx
import pandas as d

In [8]:
matrix_for_extraction = 'GP_TotTime'

# List of TAZ rows to extract
taz_rows_to_extract = [832, 1629, 1984] #832:Bountiful, 1629:Murray, 1984:South Jordan

In [9]:
# Define the input OMX file and output CSV file
omx_file = "data/input_matrix.omx"
output_csv = f"intermediate/TAZ-I-J-{matrix_for_extraction}.csv"

In [10]:
matrix_rows_to_extract = [i - 1 for i in taz_rows_to_extract]  # Convert to zero-based index

# Open the OMX file
with omx.open_file(omx_file, 'r') as omx_data:
    # Read the entire matrix_for_extraction matrix
    _matrix = omx_data[matrix_for_extraction][:]

    # Extract I-J-Value data
    extracted_data = []
    for i in matrix_rows_to_extract:
        for j in range(_matrix.shape[1]):  # Iterate over all columns
            extracted_data.append([i + 1, j + 1, _matrix[i, j]])  # Convert back to 1-based for output

# Convert to DataFrame
df = pd.DataFrame(extracted_data, columns=['I', 'J', 'Value'])

# Save to CSV
df.to_csv(output_csv, index=False)

print(f"Extracted data saved to {output_csv}")

Extracted data saved to intermediate/TAZ-I-J-GP_TotTime.csv
